## Exo 1

Passer d'un couple d'image RVB + IRC à un tenseur R,V,B,IR de même taille & compatible avec pytorch 

Etapes :

- Lire les images, les extraire en tableau numpy, concaténer & les transformer en tenseur pytorch

-	IRC : composition colorée à partir des trois canaux infrarouge, rouge et  vert (https://en.wikipedia.org/wiki/False_color) avec la correspondance de visualisation infrarouge -> rouge, rouge -> vert, vert -> bleu.

-	Un point important c'est de bien comprendre comment les données sont structurées (images en channel * height * width contre tenseur en  taille Batch * height * width * channel par exemple).

-	Pour la lecture d'image je te conseille d'utiliser la librairie rasterio (https://rasterio.readthedocs.io/en/latest/topics/reading.html)  Faire attention à l'option indexes de la méthode read

-	pas forcément utile pour ce point mais faire attention aux coordonnées/reférentiel associés à l'image (https://rasterio.readthedocs.io/en/latest/topics/georeferencing.html)

- Prise en main des données géographiques python : https://github.com/patrickcgray/open-geo-tutorial 

- Code Similaires sur datasets challenege Inria Building dataset & Spacenet :
https://medium.com/the-downlinq/object-detection-on-spacenet-5e691961d257


### Import Libraries 

In [1]:
import rasterio
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import torch
import torchvision
import torchvision.transforms as transforms
from rasterio.windows import Window

In [2]:
os.listdir('./IMAGES')

['A1_RVB', 'A2_IRC', 'A3_MNS', 'A4_MNT', 'B1_RVBIH']

In [3]:
os.listdir('./IMAGES/A1_RVB')

['readme.txt',
 'rvb.vrt',
 'zone_1.tif',
 'zone_2.tif',
 'zone_3.tif',
 'zone_4.tif',
 'zone_5.tif']

In [4]:
os.listdir('./IMAGES/A2_IRC')

['irc.vrt',
 'readme.txt',
 'zone_1.tif',
 'zone_2.tif',
 'zone_3.tif',
 'zone_4.tif',
 'zone_5.tif']

In [5]:
rasterio.gdal_version()

'3.2.2'

In [6]:
with rasterio.open('./IMAGES/A1_RVB/zone_1.tif') as src:
    print(src)

<open DatasetReader name='./IMAGES/A1_RVB/zone_1.tif' mode='r'>


### Reading Datasets 

In [7]:
def read(file):
    """
    Take as input a file and open the file with rasterio library
    
    Outputs: 
    - src.read(): numpy array of the image
    - src.profile : metadata 
    
    """
    with rasterio.open(file) as src:
        # return numpy array & meta_data
        return src.read(), src.profile

In [8]:
# Reading rvb
src_rvb = rasterio.open('./IMAGES/A1_RVB/zone_1.tif') 

# reading IRC
src_irc = rasterio.open('./IMAGES/A2_IRC/zone_1.tif') 

In [9]:
src_rvb.profile

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 30000, 'height': 20000, 'count': 3, 'crs': CRS.from_epsg(2154), 'transform': Affine(0.2, 0.0, 367000.0,
       0.0, -0.2, 6405000.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'deflate', 'interleave': 'pixel'}

In [10]:
# Geo-Trasnform 
src_rvb.profile['transform']

Affine(0.2, 0.0, 367000.0,
       0.0, -0.2, 6405000.0)

#### Extract to numpy array

In [11]:
# Extract a smaller window 256 row x 512 columns 
array_rvb = src_rvb.read(1, window=Window(0, 0, 512, 256))
array_irc = src_irc.read(1, window=Window(0, 0, 512, 256))

In [12]:
print(array_rvb)

[[ 69  60  56 ... 128  95  66]
 [ 61  61  62 ...  49  49  57]
 [ 51  51  56 ...  63  69  82]
 ...
 [ 52  52  55 ...  59  54  54]
 [ 52  52  54 ...  53  59  59]
 [ 52  55  54 ...  59  53  53]]


In [13]:
print(type(array_rvb))

<class 'numpy.ndarray'>


Height, Width 

In [14]:
array_rvb.shape

(256, 512)

In [15]:
array_irc.shape

(256, 512)

### Visualize Image

In [16]:
from rasterio.plot import show

In [17]:
#show(src_rvb)

In [18]:
#plt.imshow(src_rvb.read(),cmap='pink')
#plt.show()

We can look at the 3 axes (bands, rows, columns)

In [19]:
src_rvb.read().shape

(3, 20000, 30000)

In [20]:
src_rvb.read()[0]

array([[ 69,  60,  56, ..., 169, 167, 167],
       [ 61,  61,  62, ..., 171, 169, 170],
       [ 51,  51,  56, ..., 171, 169, 167],
       ...,
       [ 57,  60,  58, ...,  39,  39,  38],
       [ 55,  50,  47, ...,  39,  44,  46],
       [ 49,  47,  43, ...,  39,  50,  51]], dtype=uint8)

In [21]:
src_rvb.read()[0].shape

(20000, 30000)

In [22]:
src_irc.read().shape

(3, 20000, 30000)

In [23]:
from rasterio.plot import show # import the show function which allows us to display the image

#print("Image dimensions: ", src_rvb.read())
#show(array_rvb, transform=src_rvb.transform, cmap='gray')

#### Concatenate & Transform to Pytorch 

In [28]:
rvb_array = src_rvb.read(window=Window(0, 0, 512, 256))
irc_array = src_irc.read(window=Window(0, 0, 512, 256))

In [29]:
src_irc.read(window=Window(0, 0, 512, 256)).shape

(3, 256, 512)

In [30]:
src_rvb.read(window=Window(0, 0, 512, 256)).shape

(3, 256, 512)

In [31]:
rvb_array[0]

array([[ 69,  60,  56, ..., 128,  95,  66],
       [ 61,  61,  62, ...,  49,  49,  57],
       [ 51,  51,  56, ...,  63,  69,  82],
       ...,
       [ 52,  52,  55, ...,  59,  54,  54],
       [ 52,  52,  54, ...,  53,  59,  59],
       [ 52,  55,  54, ...,  59,  53,  53]], dtype=uint8)

In [34]:
rvb_array[0].shape

(256, 512)

In [35]:
irc_array[0].shape

(256, 512)

In [38]:
irc_array.shape

(3, 256, 512)

#### Concatenate RGB & IRC array

RGB -> Band (Rouge Vert Bleu)

IRC -> Band (Infrarouge Rouge Vert)

Try to concatenate RGV & IRC

In [48]:
concat_array = np.append(rvb_array,np.array([irc_array[0]]), axis = 0)

In [49]:
concat_array.shape

(4, 256, 512)

### Transform to tensor 

In [51]:
tensor_rvb_irc = torch.from_numpy(concat_array)

In [52]:
print(tensor_rvb_irc)

tensor([[[ 69,  60,  56,  ..., 128,  95,  66],
         [ 61,  61,  62,  ...,  49,  49,  57],
         [ 51,  51,  56,  ...,  63,  69,  82],
         ...,
         [ 52,  52,  55,  ...,  59,  54,  54],
         [ 52,  52,  54,  ...,  53,  59,  59],
         [ 52,  55,  54,  ...,  59,  53,  53]],

        [[ 95,  81,  75,  ..., 162, 133,  90],
         [ 89,  83,  81,  ...,  57,  56,  78],
         [ 67,  67,  71,  ...,  79,  65,  93],
         ...,
         [ 70,  69,  69,  ...,  71,  71,  71],
         [ 64,  69,  71,  ...,  76,  75,  79],
         [ 69,  71,  76,  ...,  76,  75,  78]],

        [[ 97,  88,  80,  ..., 178, 134,  86],
         [ 90,  86,  88,  ...,  57,  54,  83],
         [ 72,  86,  81,  ...,  71,  74, 115],
         ...,
         [ 77,  76,  83,  ...,  81,  85,  85],
         [ 82,  83,  82,  ...,  86,  85,  84],
         [ 82,  76,  77,  ...,  81,  81,  84]],

        [[ 17,  18,  18,  ...,  20,  19,  19],
         [ 16,  17,  17,  ...,  18,  18,  18],
         [ 1

In [53]:
tensor_rvb_irc.shape

torch.Size([4, 256, 512])